In [ ]:
import matplotlib.pyplot as plt
from matplotlib import animation
from matplotlib.collections import LineCollection
import numpy as np
import h5py

plt.rcParams['animation.ffmpeg_path'] = r'C:\Users\Christopher\Downloads\ffmpeg-n4.4-6-g7e9b9f24df-win64-gpl-4.4\ffmpeg-n4.4-6-g7e9b9f24df-win64-gpl-4.4\bin\ffmpeg.exe'
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = 'Helvetica Neue'
#sns.set_context('talk',rc={'font.family': 'sans-serif', 'font.sans-serif':['Helvetica Neue']})
ffmpeg_writer = animation.FFMpegWriter(fps=30, codec='libx264', extra_args=['-crf', '18', '-pix_fmt', 'yuv420p'])

In [ ]:
f = h5py.File('../output/julia_example_sims.h5', 'r')

In [ ]:
examples=f['tangles_full_run.002081']
print(examples.keys())
print(examples.attrs.keys())
print(examples.attrs['comment'])
print(examples.attrs['rates.topo'])
print(examples.attrs['gene.base_rate'])
print(examples['time'].shape)
print(examples.attrs['rates.sc_dependent'])
print(examples.attrs['bcs.is_circular'])

In [ ]:
print(examples['rnap_location'][:,150:200])
print(examples['time'][150:200])

In [ ]:
def interpolate(data_group, time):
    """
    Taking a HDF5 group and a time to interpolate at, returns a tuple representing an
    interpolated state. mRNA state is interpolated "from left" (e.g. constant), whereas
    other variables are linearly interpolated. If the two nearest points differ in # of
    polymerases, the "left" state is used without modification.
    """
    idx = 0
    total_points = len(data_group['time'])
    times = np.array(data_group['time'][:])
    while times[idx] < time and idx < total_points:
        idx = idx + 1
    if idx == total_points:
        return {"rnap_location": [], 'phi': [], 'mRNA_length': [], 'mRNA': []}
    
    if idx == total_points - 1:
        # Just return the last point
        return {k:data_group[k][:,idx] for k in ['rnap_location', 'phi', 'mRNA_length', 'mRNA']}

    # Calc number of polymerases at timepoint i and i + 1
    prev_n_rnap = sum(data_group['rnap_location'][:,idx-1] < 0)
    next_n_rnap = sum(data_group['rnap_location'][:,idx] < 0)

    if prev_n_rnap != next_n_rnap:
        # Just return the last point
        return {k:data_group[k][:,idx-1] for k in ['rnap_location', 'phi', 'mRNA_length', 'mRNA']}

    # Otherwise, now interpolate everything except the mRNA values
    alpha = (data_group['time'][idx] - time) / (data_group['time'][idx] - data_group['time'][idx - 1])
    return {
        'rnap_location': (data_group['rnap_location'][:,idx-1] * alpha) + (data_group['rnap_location'][:,idx] * (1 - alpha)),
        'phi': (data_group['phi'][:,idx-1] * alpha) + (data_group['phi'][:,idx] * (1 - alpha)),
        'mRNA_length': (data_group['mRNA_length'][:,idx-1] * alpha) + (data_group['mRNA_length'][:,idx] * (1 - alpha)),
        'mRNA': data_group['mRNA'][:,idx-1]
    }

def gen_animation(data_group, gene_colors, filename, n_frames, tspan=None):
    is_circular = data_group.attrs['bcs.is_circular'] == 1.0

    genes = list(zip(data_group.attrs['gene.start'], data_group.attrs['gene.end']))

    times = np.array(data_group['time'])
    mRNA_data = np.array(data_group['mRNA'])
    #max_time = max(data_group['time'])
    if tspan is None:
        tspan = (0, max(data_group['time']))
    max_length = data_group.attrs['bcs.length']
    max_gene_length = max([abs(gene[1] - gene[0]) for gene in genes])
    to_theta = lambda x: 2*np.pi * x / max_length

    fig = plt.figure(figsize=(1920 / 300,1000 / 300))
    ax1 = plt.subplot(1, 3, (1, 2), polar=True) if is_circular else plt.subplot(1, 3, (1,2))
    ax2 = plt.subplot(1, 3, 3)

    if not is_circular:
        ax1.set_xlim([0,max_length])
        ax1.set_ylim([-20, 100 + max_gene_length])
        ax1.spines['top'].set_visible(False)
        ax1.spines['right'].set_visible(False)
        ax1.spines['left'].set_visible(False)
        ax1.tick_params(left=False, labelleft=False)
        ax1.set_xlabel('Distance (nm)')
    else:
        ax1.tick_params(left=False, labelleft=False, bottom=False, labelbottom=False)
        ax1.set_ylim(0,2)
        #ax1.set_rmax(3)
        ax1.grid(False)
        ax1.spines['polar'].set_visible(False)

    ax2.set_xlim(tspan)
    ax2.set_ylim([0,np.max(data_group['mRNA'])])
    ax2.spines['top'].set_visible(False)
    ax2.spines['right'].set_visible(False)
    ax2.set_xlabel('Time (s)')
    ax2.set_ylabel('# mRNAs')

    gene_lines = [ax2.plot([], [], color=color)[0] for gene, color in zip(genes, gene_colors)]

    polymerases = ax1.plot([], [], 'ko')[0]

    if is_circular:
        gene_plots = [ax1.plot(np.linspace(to_theta(gene[0]), to_theta(gene[1]), 50), np.linspace(1,1,50), linewidth=8, zorder=0, color=color)[0] for gene, color in zip(genes, gene_colors)]
        ax1.plot(np.linspace(0, 2 * np.pi, 50), np.linspace(1,1,50), linewidth=4, zorder=0, color=plt.cm.coolwarm(0.5), solid_capstyle='butt')
    else:
        gene_plots = [ax1.plot(gene, [0, 0], color=color,linewidth=14, zorder=0, solid_capstyle='butt')[0] for gene, color in zip(genes, gene_colors)]
        ax1.plot([0, max_length], [0, 0], color=plt.cm.coolwarm(0.5), linewidth=8, zorder=0, solid_capstyle='butt')

    main_line = LineCollection([], cmap=plt.cm.coolwarm, norm=plt.Normalize(-.2,.2))
    mRNA_lines = LineCollection([], colors='k')

    def init():
        main_line.set_segments([])
        if is_circular:
            main_line.set_linewidth(4)
        else:
            main_line.set_linewidth(8)
        mRNA_lines.set_linewidth(3)
        ax1.add_collection(main_line)
        ax1.add_collection(mRNA_lines)
        return (*gene_plots, main_line, mRNA_lines)
    
    def animate(i):
        t = tspan[0] + (tspan[1] - tspan[0]) * (i / n_frames)
        frame_vals = interpolate(data_group, t)
        n_polymerases = np.sum(frame_vals['rnap_location'] >= 0)

        # Plot mRNA data values

        # Plot mRNA values (on the right-hand plot)
        gene_time_mask = (times < t)
        for i, gene_line in enumerate(gene_lines):
            gene_line.set_data(times[gene_time_mask], mRNA_data[i,gene_time_mask])

        # Set visibility
        polymerases.set_visible(n_polymerases > 0)
        main_line.set_visible(n_polymerases > 0)
        mRNA_lines.set_visible(n_polymerases > 0)

        # Plot polymerases and genes coming off polymerases.
        if n_polymerases > 0:
            if is_circular:
                # Simulate circular boundary conditions
                rnap_locs = np.concatenate((
                    np.ones(1,) * (frame_vals['rnap_location'][max(0,n_polymerases - 1)] - max_length),
                    frame_vals['rnap_location'][:n_polymerases],
                    np.ones(1,) * (max_length + frame_vals['rnap_location'][0])
                ))
                phi = np.concatenate((
                    np.ones(1,) * frame_vals['phi'][max(0,n_polymerases - 1)],
                    frame_vals['phi'][:n_polymerases],
                    np.ones(1,) * frame_vals['phi'][0]
                ))
                sigma = np.diff(phi) / np.diff(rnap_locs) / -1.85


                theta_rnap = to_theta(rnap_locs)
                theta_interp = np.linspace(0, 2 * np.pi, 200)
                sigma_interp = np.interp(theta_interp,
                    np.concatenate((
                        np.array([theta_rnap[1]]),
                        theta_rnap[1:-2],
                        np.array([theta_rnap[-2]]))), sigma)
                points = np.array([theta_interp, np.ones(theta_interp.shape) * 1]).T.reshape(-1,1,2)
                segments = np.concatenate([points[:-1], points[1:]], axis=1)
                main_line.set_segments(segments)
                main_line.set_array(sigma_interp)
                polymerases.set_data(theta_rnap[1:-1], np.ones_like(theta_rnap[1:-1]))
                mRNA_base = np.array([to_theta(rnap_locs[1:-1]), np.ones(rnap_locs[1:-1].shape)]).T.reshape(-1,1,2)
                mRNA_tail = np.array([to_theta(rnap_locs[1:-1]), 1 + (frame_vals['mRNA_length'][:n_polymerases] / max_gene_length)]).T.reshape(-1,1,2)
                mRNA_segments = np.concatenate([mRNA_base, mRNA_tail], axis=1)
                mRNA_lines.set_segments(mRNA_segments)
            else:
                rnap_locs = np.concatenate((
                    np.zeros(1,),
                    frame_vals['rnap_location'][:n_polymerases],
                    np.ones(1,) * max_length
                ))
                phi = np.concatenate((
                    np.zeros(1,),
                    frame_vals['phi'][:n_polymerases],
                    np.zeros(1,)
                ))
                sigma = np.diff(phi) / np.diff(rnap_locs) / -1.85

                points = np.array([rnap_locs, np.zeros_like(rnap_locs)]).T.reshape(-1,1,2)
                segments = np.concatenate([points[:-1], points[1:]], axis=1)
                main_line.set_segments(segments)
                main_line.set_array(sigma)
                polymerases.set_data(rnap_locs[1:-1], np.zeros_like(rnap_locs[1:-1]))
                mRNA_base = np.array([rnap_locs[1:-1], np.zeros(rnap_locs[1:-1].shape)]).T.reshape(-1,1,2)
                mRNA_tail = np.array([rnap_locs[1:-1], frame_vals['mRNA_length'][:n_polymerases]]).T.reshape(-1,1,2)
                mRNA_segments = np.concatenate([mRNA_base, mRNA_tail], axis=1)
                mRNA_lines.set_segments(mRNA_segments)
        return [main_line, polymerases, mRNA_lines] + list(gene_lines)
    anim = animation.FuncAnimation(fig, animate, init_func=init, frames=n_frames, interval=33, blit=True)
    anim.save(filename, dpi=300, writer=ffmpeg_writer, progress_callback= lambda i,n: print(f'Saving frame {i} of {n}') if i % 250 == 0 else None)

In [ ]:
max([f[run].attrs['gene.base_rate'][0] for run in f])

In [ ]:
for run in f:
    cur_run = f[run]
    if cur_run.attrs['rates.sc_dependent'] != 1.0:
        continue
    if cur_run.attrs['comment'].decode('utf8') != '2_gene.divergent':
        continue
    if cur_run.attrs['bcs.is_circular'] != 0:
        continue
    if not (cur_run.attrs['gene.base_rate'] > 0.008).all():
        continue
    print(run)

In [ ]:
gen_animation(f['tangles_full_run.001151'], ['#808080', '#A58245'], '../output/divergent_plasmid_closeup.mp4', 1000, (6800,7200))
gen_animation(f['tangles_full_run.001151'], ['#808080', '#A58245'], '../output/divergent_plasmid.mp4', 2000, (5000,10000))
gen_animation(f['tangles_full_run.001749'], ['#808080', '#A58245'], '../output/convergent_plasmid.mp4', 2000, (6000,8000))
gen_animation(f['tangles_full_run.001744'], ['#808080', '#A58245'], '../output/tandem_plasmid.mp4', 2000, (3000, 6000))
gen_animation(f['tangles_full_run.000246'], ['#808080', '#A58245'], '../output/convergent_linear_switch.mp4', 4000, (0, 7000))

In [ ]:
gen_animation(f['tangles_full_run.001151'], ['#808080', '#A58245'], '../output/divergent_plasmid_slowmo.mp4', 4000, (7000,7200))

In [ ]:
gen_animation(f['tangles_full_run.002285'], ['#808080', '#A58245'], '../output/tandem_relative_Rates.mp4', 250)

In [ ]:
gen_animation(f['tangles_full_run.002081'], ['#808080', '#A58245'], '../output/divergent_linear.mp4', 1500, (2000,6000))